# A股量化交易系统 - 快速入门

本笔记本演示如何使用量化交易系统进行数据获取、因子计算和策略回测。

## 1. 环境配置

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 导入系统模块
from src.data import DataFetcher, get_stock_data
from src.factors import FactorEngine
from src.backtest import BacktestEngine, BacktestConfig
from src.strategy import MACrossStrategy, MomentumStrategy

## 2. 数据获取

In [ ]:
# 初始化数据获取器
fetcher = DataFetcher(source='akshare')

# 获取股票列表
stock_list = fetcher.get_stock_list()
print(f"共 {len(stock_list)} 只股票")
stock_list.head(10)

In [ ]:
# 获取单只股票日线数据
code = '000001'  # 平安银行
df = fetcher.get_daily_data(code, start_date='2024-01-01')
print(f"{code} 数据量: {len(df)} 条")
df.tail()

In [ ]:
# 绘制K线图
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), height_ratios=[3, 1])

# 价格走势
ax1.plot(df.index, df['close'], label='收盘价', linewidth=1.5)
ax1.fill_between(df.index, df['low'], df['high'], alpha=0.3)
ax1.set_title(f'{code} 股价走势', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# 成交量
colors = ['g' if df['close'].iloc[i] >= df['open'].iloc[i] else 'r' for i in range(len(df))]
ax2.bar(df.index, df['volume'], color=colors, alpha=0.7)
ax2.set_title('成交量', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 因子计算

In [ ]:
# 初始化因子引擎
factor_engine = FactorEngine()

# 查看可用因子
print("可用因子:")
for name in factor_engine.factors.keys():
    print(f"  - {name}")

In [ ]:
# 计算因子
factors_to_compute = ['momentum_5', 'momentum_20', 'ma_20', 'ma_60', 'rsi_14', 'volatility_20']
df_with_factors = factor_engine.compute(df, factors_to_compute)
df_with_factors.tail(10)

In [ ]:
# 可视化因子
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# 价格和均线
axes[0].plot(df_with_factors.index, df_with_factors['close'], label='收盘价')
axes[0].plot(df_with_factors.index, df_with_factors['ma_20'], label='MA20', linestyle='--')
axes[0].plot(df_with_factors.index, df_with_factors['ma_60'], label='MA60', linestyle='--')
axes[0].legend()
axes[0].set_title('价格与均线')
axes[0].grid(True, alpha=0.3)

# RSI
axes[1].plot(df_with_factors.index, df_with_factors['rsi_14'], color='purple')
axes[1].axhline(y=70, color='r', linestyle='--', alpha=0.5)
axes[1].axhline(y=30, color='g', linestyle='--', alpha=0.5)
axes[1].fill_between(df_with_factors.index, 30, 70, alpha=0.1)
axes[1].set_title('RSI(14)')
axes[1].set_ylim(0, 100)
axes[1].grid(True, alpha=0.3)

# 动量
axes[2].bar(df_with_factors.index, df_with_factors['momentum_20'], alpha=0.7)
axes[2].axhline(y=0, color='black', linewidth=0.5)
axes[2].set_title('20日动量')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. 策略回测

In [ ]:
# 准备多只股票数据
codes = ['000001', '000002', '600000', '600036', '601398']
data = {}
for code in codes:
    try:
        data[code] = fetcher.get_daily_data(code, start_date='2024-01-01')
        print(f"{code}: {len(data[code])} 条数据")
    except Exception as e:
        print(f"{code}: 获取失败 - {e}")

In [ ]:
# 配置回测
config = BacktestConfig(
    initial_capital=1_000_000,  # 100万初始资金
    commission_rate=0.0003,      # 万三佣金
    stamp_duty=0.001,            # 千一印花税
    slippage=0.001               # 0.1%滑点
)

# 创建回测引擎
engine = BacktestEngine(config)

In [ ]:
# 使用均线交叉策略
strategy = MACrossStrategy(short_period=5, long_period=20)

# 运行回测
result = engine.run(data, strategy)

# 输出结果
print("\n" + "="*50)
print("均线交叉策略回测结果")
print("="*50)
for k, v in result.summary().items():
    print(f"{k}: {v}")

In [ ]:
# 绘制收益曲线
if result.equity_curve is not None and len(result.equity_curve) > 0:
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), height_ratios=[3, 1])
    
    # 权益曲线
    returns = result.equity_curve / result.equity_curve.iloc[0] - 1
    axes[0].plot(returns.index, returns * 100, label='策略收益', linewidth=1.5)
    axes[0].set_title('策略收益曲线', fontsize=14)
    axes[0].set_ylabel('收益率 (%)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 回撤曲线
    rolling_max = result.equity_curve.cummax()
    drawdown = (result.equity_curve - rolling_max) / rolling_max * 100
    axes[1].fill_between(drawdown.index, drawdown, 0, alpha=0.5, color='red')
    axes[1].set_title('回撤曲线', fontsize=12)
    axes[1].set_ylabel('回撤 (%)')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("没有足够的回测数据")

## 5. 策略对比

In [ ]:
# 对比多个策略
strategies = [
    ('均线交叉(5/20)', MACrossStrategy(5, 20)),
    ('均线交叉(10/30)', MACrossStrategy(10, 30)),
    ('动量策略(Top3)', MomentumStrategy(lookback=20, top_n=3)),
]

results = []
for name, strategy in strategies:
    engine = BacktestEngine(config)
    result = engine.run(data, strategy)
    result.name = name
    results.append(result)
    print(f"{name}: 收益率 {result.total_return:.2%}, 夏普 {result.sharpe_ratio:.2f}")

In [ ]:
# 可视化对比
fig, ax = plt.subplots(figsize=(14, 6))

for result in results:
    if result.equity_curve is not None and len(result.equity_curve) > 0:
        returns = result.equity_curve / result.equity_curve.iloc[0] - 1
        ax.plot(returns.index, returns * 100, label=result.name, linewidth=1.5)

ax.set_title('策略收益对比', fontsize=14)
ax.set_xlabel('日期')
ax.set_ylabel('收益率 (%)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 下一步

- 探索更多因子: 查看 `src/factors/factor_engine.py` 中的因子库
- 开发自己的策略: 继承 `BaseStrategy` 类
- 集成机器学习: 使用 Qlib 进行 AI 选股